In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn import set_config
from sklearn import datasets
from sklearn import ensemble, neighbors,linear_model,svm
import eli5
set_config(display='diagram')
datasets.load*?

datasets.load_boston
datasets.load_breast_cancer
datasets.load_diabetes
datasets.load_digits
datasets.load_files
datasets.load_iris
datasets.load_linnerud
datasets.load_sample_image
datasets.load_sample_images
datasets.load_svmlight_file
datasets.load_svmlight_files
datasets.load_wine

In [72]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [82]:
df = sklearn_to_df(datasets.load_breast_cancer())

numeric_features = df.select_dtypes(include=['int64', 'float64']).drop(['target'], axis=1).columns
categorical_features = df.select_dtypes(include=['object']).columns

X = df.drop('target',axis = 'columns')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state =1)

In [83]:
# train_values = pd.read_csv('train_values.csv')
# train_labels = pd.read_csv('train_labels.csv')
# train_data = train_values.merge(train_labels, left_on='building_id', right_on='building_id')

# train_data = train_data.drop('building_id', axis=1)
# numeric_features = train_data.select_dtypes(include=['int64', 'float64']).drop(['damage_grade'], axis=1).columns
# categorical_features = train_data.select_dtypes(include=['object']).columns
# X = train_data.drop('damage_grade', axis=1)
# y = train_data['damage_grade']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [86]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn import ensemble, neighbors,linear_model,svm
from sklearn.feature_selection import SelectFromModel

numeric_transformer = Pipeline(steps=[
    #('feature_selection',SelectFromModel(svm.LinearSVC(penalty = 'l1',dual = False))),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot', OneHotEncoder())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model',  ensemble.RandomForestRegressor(max_depth = 3))])
    
#model = pipe.fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'tex...
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder())]),
                                                  Index([], dtype='object'))])),
                ('model', RandomForestRegressor(max_depth=3))])

In [90]:
try:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestRegressor(max_depth = 3))])
    pipe.fit(X_train,y_train)
except:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('model',  ensemble.RandomForestClassifier(max_depth = 3))])
    pipe.fit(X_train,y_train)

try:
    onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot']
                          .get_feature_names(input_features=categorical_features))
except:
    onehot_columns = []

numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)
eli5.explain_weights(pipe.named_steps['model'], top=10, feature_names=numeric_features_list)

Weight,Feature
0.5727 ± 0.7499,worst perimeter
0.2174 ± 0.6771,worst concave points
0.0480 ± 0.3224,worst area
0.0337 ± 0.0806,worst texture
0.0293 ± 0.2255,mean concave points
0.0249 ± 0.2331,worst radius
0.0125 ± 0.0469,mean texture
0.0102 ± 0.0505,mean concavity
0.0075 ± 0.0305,worst smoothness
0.0067 ± 0.0306,area error


In [91]:
model_list = [#regressors
                linear_model.LinearRegression(),linear_model.Lasso(),linear_model.Ridge(),
              
              #ensembling - regressors
                ensemble.RandomForestRegressor(max_depth = 3),ensemble.AdaBoostRegressor(),ensemble.GradientBoostingRegressor(),
    
              #classifiers
                linear_model.LogisticRegression(max_iter = 500), neighbors.KNeighborsClassifier(),svm.SVC(),             
    
              #ensembling - classifiers
                ensemble.RandomForestClassifier(n_estimators=500,max_depth = 3),ensemble.AdaBoostClassifier(learning_rate=0.5),ensemble.GradientBoostingClassifier()

              ]

In [92]:
rejected_models = []
results = []
print('{:30s}Train  /  Test'.format('Model'))
print('---------------------------------------------')
for model in model_list:
    #pipe = make_pipeline(preprocessor,model)
    #pipe = make_pipeline(model)
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])
    
    try:        
        pipe.fit(X_train,y_train)
        
        model_name = type(model).__name__
        train_results = pipe.score(X_train,y_train)
        test_results = pipe.score(X_test,y_test)
        
        results.append([model_name, train_results, test_results])
        print('{:30s}{:.4f} / {:.4f} '.format(model_name, train_results, test_results))

    except:
        rejected_models.append(type(model).__name__)

df_results = pd.DataFrame(results, columns=["Model", "Train Results","Test Results"])
print('Rejected Models : {}'.format(rejected_models))

Model                         Train  /  Test
---------------------------------------------
LinearRegression              0.7798 / 0.7394 
Lasso                         0.0000 / -0.0002 
Ridge                         0.7768 / 0.7412 
RandomForestRegressor         0.9480 / 0.7758 
AdaBoostRegressor             0.9975 / 0.8408 
GradientBoostingRegressor     0.9979 / 0.8396 
LogisticRegression            0.9899 / 0.9708 
KNeighborsClassifier          0.9799 / 0.9591 
SVC                           0.9899 / 0.9708 
RandomForestClassifier        0.9925 / 0.9415 
AdaBoostClassifier            1.0000 / 0.9474 
GradientBoostingClassifier    1.0000 / 0.9591 
Rejected Models : []


In [93]:
df_results.sort_values(by = 'Test Results',ascending = False)

,Model,Train Results,Test Results
6,LogisticRegression,0.989950,0.970760
8,SVC,0.989950,0.970760
7,KNeighborsClassifier,0.979899,0.959064
11,GradientBoostingClassifier,1.000000,0.959064
10,AdaBoostClassifier,1.000000,0.947368
9,RandomForestClassifier,0.992462,0.941520
4,AdaBoostRegressor,0.997498,0.840767
5,GradientBoostingRegressor,0.997903,0.839551
3,RandomForestRegressor,0.947954,0.775830
2,Ridge,0.776811,0.741223
